In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate

In [16]:
path = "../training/conf/network/convnext.yaml"

In [37]:
with open(path, "rb") as f:
    cfg = OmegaConf.load(f)

In [38]:
cfg

{'_target_': 'text_recognizer.network.convnext.ConvNext', 'dim': 16, 'dim_mults': [2, 4, 8], 'depths': [3, 3, 6], 'downsampling_factors': [[2, 2], [2, 2], [2, 2]], 'attn': {'_target_': 'text_recognizer.network.convnext.TransformerBlock', 'attn': {'_target_': 'text_recognizer.network.convnext.Attention', 'dim': 128, 'heads': 4, 'dim_head': 64, 'scale': 8}, 'ff': {'_target_': 'text_recognizer.network.convnext.FeedForward', 'dim': 128, 'mult': 4}}}

In [19]:
cfg.dim_mults = [2, 4, 8, 8]
cfg.depths = [3, 3, 6, 6]
cfg.downsampling_factors = [[2, 2], [2, 2], [2, 2], [2, 1]]

In [39]:
net = instantiate(cfg)

In [40]:
from torchinfo import summary

In [41]:
summary(net, (2, 1, 576, 640), device="cpu", depth=4)

Layer (type:depth-idx)                             Output Shape              Param #
ConvNext                                           [2, 128, 72, 80]          165,408
├─TransformerBlock: 1-5                            [2, 128, 72, 80]          (recursive)
│    └─Attention: 2-6                              [2, 128, 72, 80]          (recursive)
│    │    └─LayerNorm: 3-13                        [2, 128, 72, 80]          (recursive)
├─Conv2d: 1-3                                      [2, 16, 576, 640]         (recursive)
├─TransformerBlock: 1                              --                        --
│    └─Attention: 2                                --                        --
│    │    └─Conv2d: 3-15                           [2, 128, 72, 80]          (recursive)
│    └─FeedForward: 2-7                            [2, 128, 72, 80]          (recursive)
│    │    └─Residual: 3-16                         [2, 128, 72, 80]          (recursive)
│    │    │    └─Sequential: 4-26              